# SK RAG Pattern Foundational Concepts - Redis Cache

Learning objectives:

- RAG pattern foundational concepts
- Redis cache as a vector database

## Redis setup

### Running Redis in a local container:

- docker pull `redis/redis-stack:latest`
  - **Note:** this version of redis includes the `RedisSearch` module
- Then execute: 
  - `docker run -d --name redis-stack -p 6379:6379 -p 8001:8001 redis/redis-stack:latest`

Connection string:

- `REDIS_CONN_STR=localhost`

### Running from Azure Redis Cache



## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.4.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Redis, 1.4.0-alpha"
#r "nuget: StackExchange.Redis"
#r "nuget: dotenv.net"

using System;

using System.IO;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using StackExchange.Redis;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Connectors.Redis;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;

using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Models/Models.cs

const int ADA_EMBEDDING_SIZE = 1536;
const string MemoryCollectionName = "SemanticCache";

Installed Packages dotenv.net, 3.1.3 Microsoft.SemanticKernel, 1.4.0 Microsoft.SemanticKernel.Connectors.Redis, 1.4.0-alpha Microsoft.SemanticKernel.Core, 1.4.0 Microsoft.SemanticKernel.Plugins.Memory, 1.4.0-alpha StackExchange.Redis, 2.7.17

### Read the API Key and endpoints from environment variables or the .env file

In [2]:
// Load the .env file
DotEnv.Load();

// Get the OpenAI deployment name, endpoint, and key from the environment variables
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var redis_conn_str = Environment.GetEnvironmentVariable("REDIS_CONN_STR");
var adaDeploymentName = "ada";

### Get a kernel instance configured for text completions and embeddings

In [3]:
// I'm using a RAM stored Vector DB, but I can switch providers like Azure Search, DuckDB, SQLite, etc.
#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey)
    .AddAzureOpenAITextEmbeddingGeneration(adaDeploymentName, endpoint, apiKey)
    .Build();

In [4]:
#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102

ConnectionMultiplexer connectionMultiplexer = await ConnectionMultiplexer.ConnectAsync(redis_conn_str);
IDatabase database = connectionMultiplexer.GetDatabase();
var memoryStore = new RedisMemoryStore(database, vectorSize: ADA_EMBEDDING_SIZE);

// Reset the collection
if (await memoryStore.DoesCollectionExistAsync(MemoryCollectionName))
    await memoryStore.DeleteCollectionAsync(MemoryCollectionName);

var embeddingGenerator = new AzureOpenAITextEmbeddingGenerationService(adaDeploymentName, endpoint, apiKey);

// The combination of the text embedding generator and the memory store makes up the 'SemanticTextMemory' object used to
// store and retrieve memories.
SemanticTextMemory textMemory = new(memoryStore, embeddingGenerator);

### Function to read and save to and from Cache

In [5]:
async Task<Chunk?> CheckInCache(string question, double minRelevance=0.8)
{
    try 
    {
        Chunk? chunk = null;    
        #pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
        IAsyncEnumerable<MemoryQueryResult> queryResults =
                        textMemory.SearchAsync(MemoryCollectionName, question, limit: 1, minRelevanceScore: minRelevance);
        await foreach (MemoryQueryResult r in queryResults)
        {
            chunk = new Chunk(r.Metadata.Id, r.Metadata.Text, string.Empty);
        }
        return chunk;
    }
    catch (Exception)
    {
        return null;
    }
}

async Task SaveInCache(Chunk chunk)
{
    await textMemory.SaveInformationAsync(MemoryCollectionName, id: chunk.Id, text: chunk.Text);
}

### Call the LLM

In [6]:
async Task<string> CallLLM(string prompt, int maxTokens = 500, double temperature = 0.3)
{
    var skfunc = kernel.CreateFunctionFromPrompt(prompt, new OpenAIPromptExecutionSettings() { MaxTokens = maxTokens, Temperature = temperature, TopP = 1 });
    var result = await kernel.InvokeAsync(skfunc);
    return result.ToString();
}

### Prompt the completion looking at the Cache results first

In [7]:
async Task ProcessPrompt(string prompt)
{
    var chunk = await CheckInCache(prompt);
    if (chunk is null)
    {
        // Handle Cache miss
        Console.WriteLine($"Cache miss:\nuser: {prompt}");
        var result = await CallLLM(prompt);
        Console.WriteLine($"Calling the LLM\nResult: {result}");
        chunk = new Chunk(Guid.NewGuid().ToString(), result, string.Empty);
        Console.WriteLine($"Adding the result to cache.");
        await SaveInCache(chunk);
    }
    else
    {
        Console.WriteLine($"Cache hit:");
        Console.WriteLine($"user: {prompt}");
        Console.WriteLine($"assitant: {chunk.Text}");
    }
}


### Cache the results of the Prompt

In [8]:
await ProcessPrompt("What is the speed of light?");

Cache miss:
user: What is the speed of light?
Calling the LLM
Result: The speed of light in a vacuum is approximately 299,792,458 meters per second (or about 186,282 miles per second). This is denoted by the symbol "c" in physics equations.
Adding the result to cache.


### Change the Prompt slightly and try to get the answer from cache

In [9]:
await ProcessPrompt("State the speed of light.");

Cache hit:
user: State the speed of light.
assitant: The speed of light in a vacuum is approximately 299,792,458 meters per second (or about 186,282 miles per second). This is denoted by the symbol "c" in physics equations.
